<a href="https://colab.research.google.com/github/drdww/OPIM5641/blob/main/Module3/M3_1/Allocation_Models_25percent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Programming: Allocation Models (25% constraint)

**OPIM 5641: Business Decision Modeling - Dept. of Operations and Information Management - University of Connecticut**

---------------------

Let's add a new constraint that each furniture line must represent 25% of the total inventory produced! 

Related Readings:
* `Pyomo Cookbook`: https://github.com/jckantor/ND-Pyomo-Cookbook/blob/master/notebooks/02.01-Production-Models-with-Linear-Constraints.ipynb
* `Powell`: Chapter 9 (Linear Optimization)



## Setup Your Environment/Imports

In [ ]:
# before you do anything...
# mount your drive!
# click folder on the left...

In [ ]:
# import modules

%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

from pyomo.environ import *

     |████████████████████████████████| 9.0MB 2.7MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 256kB 45.4MB/s 
     |████████████████████████████████| 163kB 45.9MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 144487 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coi

# Example: Furniture
*Section 9.2 (Powell) - Chairs, Desks and Tables*

**Problem Description:**
Veerman Furniture Company makes three kinds of ofﬁce furniture: chairs, desks, and tables. Each product requires some labor in the parts fabrication department, the assembly department, and the shipping department. The furniture is sold through a regional distributor, who has estimated the maximum potential sales for each product in the coming quarter. Finally,the accounting department has provided some data showing the proﬁt contributions on each product.The decision problem isto determine the product mix—that is, to maximize Veerman’s proﬁt for the quarter by choosing production quantities for the chairs,desks,and tables.

The following data summarizes the parameters of the problem:

Department | Chairs | Desks | Tables | Hours Available
--- | --- | --- | --- | ---
Fabrication | 4 | 6 | 2 | 1,850
Assembly | 3 | 5 | 7 | 2,400
Shipping | 3 | 2 | 4 | 1,500
--------------------------------------------------------------------------------
Demand Potential | 360 | 300 | 100 |
Profit (USD) | 15 | 24 | 18 |

**Change the model so as to consider the fact that each product must encompass at least 25% of the total amount of furniture (i.e., number of items) made.** 


**Define the Objective Function**

$Profit = 15C + 24D + 18T$

**Write the Constraints**

$Max(Z) = 15C + 24D + 18T$

subject to:
* $4C + 6D + 2T <= 1,850$
* $3C + 5D + 7T <= 2,400$
* $3C + 2D + 4T <= 1,500$ 
* $C        <= 360$ 
* $D      <=300$ 
* $T <=10$

new constraints:

* $C >= 0.25(C + D + T)$
* $D >= 0.25(C + D + T)$
* $T >= 0.25(C + D + T)$


In [ ]:
# declare the model
model = ConcreteModel()

# declare decision variables
model.c = Var(domain=NonNegativeReals)
model.d = Var(domain=NonNegativeReals)
model.t = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(
                      expr = 15*model.c + 24*model.d + 18*model.t, # values come from the table
                      sense = maximize)

# declare constraints
model.Constraint1 = Constraint(expr = 4*model.c + 6*model.d + 2*model.t <= 1850) # fabrication hours
model.Constraint2 = Constraint(expr = 3*model.c + 5*model.d + 7*model.t <= 2400) # assembly hours
model.Constraint3 = Constraint(expr = 3*model.c + 2*model.d + 4*model.t <= 1500) # shipping
model.Constraint4 = Constraint(expr = model.c <= 360) # c demand
model.Constraint5 = Constraint(expr = model.d <= 300) # d demand
model.Constraint6 = Constraint(expr = model.t <= 100) # t demand
model.Constraint7 = Constraint(expr = model.c >= 0.25*(model.c + model.d + model.t)) 
model.Constraint8 = Constraint(expr = model.d >= 0.25*(model.c + model.d + model.t)) 
model.Constraint9 = Constraint(expr = model.t >= 0.25*(model.c + model.d + model.t)) 


In [ ]:
# show the model you've created
model.pprint()

3 Var Declarations
    c : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    d : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    t : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 15*c + 24*d + 18*t

9 Constraint Declarations
    Constraint1 : Size=1, Index=None, Active=True
        Key  : Lower : Body            : Upper  : Active
        None :  -Inf : 4*c + 6*d + 2*t : 1850.0 :   True
    Constraint2 : Size=1, Index=None, Active=True
        Key  : Lower : Body            : Upper  : Active
        None :  -Inf : 3*c + 

In [ ]:
# ensure you have cbc installed
# !apt-get install -y -qq coinor-cbc

In [ ]:
# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 8100.0
  Upper bound: 8100.0
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 4
  Number of nonzeros: 3
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created 

In [ ]:
# show the results
print("Profit = ", model.profit(), " per week")
print("Chairs = ", model.c(), " units per week")
print("Desks = ", model.d(), " units per week")
print("Tables = ", model.t(), " units per week")

Profit =  8100.0  per week
Chairs =  100.0  units per week
Desks =  200.0  units per week
Tables =  100.0  units per week
